In [1]:
# !pip install pennylane-catalyst torch torchvision

# Additional imports for hybrid approach
import pennylane as qml
import numpy as np
from itertools import combinations
from typing import List, Optional, Sequence, Tuple
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_squared_error
from copy import deepcopy
from tqdm.auto import tqdm
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import time
from tqdm.notebook import tqdm
import pennylane as qml
from sklearn.metrics import r2_score, mean_squared_error

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"PyTorch device: {device}")
print(f"CUDA available: {torch.cuda.is_available()}")

# Try importing Catalyst
try:
    from catalyst import qjit
    print("Catalyst available: True")
    CATALYST_AVAILABLE = True
except ImportError:
    print("Catalyst available: False (JIT features will be disabled)")
    CATALYST_AVAILABLE = False

PyTorch device: cuda
CUDA available: True
Catalyst available: True


In [2]:
class ImprovedNeuralReadout(nn.Module):
    """
    Enhanced feedforward neural network for quantum readout.
    Improvements: BatchNorm, LeakyReLU, better regularization.
    """
    def __init__(self, input_size, hidden_layers=[256, 128, 64], dropout=0.3):
        super().__init__()
        layers = []
        prev_size = input_size

        for i, hidden_size in enumerate(hidden_layers):
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.LeakyReLU(0.1),
                nn.Dropout(dropout)
            ])
            prev_size = hidden_size

        layers.append(nn.Linear(prev_size, 1))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x).squeeze(-1)


class AttentionReadout(nn.Module):
    """
    Attention-based readout that learns reservoir importance.
    """
    def __init__(self, input_size, n_reservoirs=3, hidden_size=128, num_heads=4):
        super().__init__()
        self.n_reservoirs = n_reservoirs
        self.obs_per_reservoir = input_size // n_reservoirs

        # Multi-head attention
        self.attention = nn.MultiheadAttention(
            embed_dim=self.obs_per_reservoir,
            num_heads=num_heads,
            batch_first=True,
            dropout=0.1
        )

        # Prediction network
        self.predictor = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        batch_size = x.shape[0]

        # Reshape to (batch, n_reservoirs, obs_per_reservoir)
        x_reshaped = x.view(batch_size, self.n_reservoirs, -1)

        # Apply attention across reservoirs
        attn_output, attn_weights = self.attention(x_reshaped, x_reshaped, x_reshaped)

        # Flatten and predict
        return self.predictor(attn_output.flatten(1)).squeeze(-1)


print("Neural readout classes defined successfully")

Neural readout classes defined successfully


In [3]:
class GPUTrainer:
    """
    GPU-accelerated trainer with early stopping, learning rate scheduling.
    """
    def __init__(self, model, device='cuda', lr=0.001, weight_decay=1e-4):
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')
        self.model = model.to(self.device)

        # AdamW optimizer (better than Adam)
        self.optimizer = optim.AdamW(
            model.parameters(),
            lr=lr,
            betas=(0.9, 0.999),
            weight_decay=weight_decay
        )

        # Learning rate scheduler
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer,
            mode='min',
            factor=0.5,
            patience=10,
            min_lr=1e-6
        )

        self.criterion = nn.MSELoss()
        self.best_model_state = None
        self.training_history = {'train_loss': [], 'val_loss': []}

    def train_epoch(self, X_train, y_train, batch_size=32):
        self.model.train()
        total_loss = 0
        n_batches = 0

        # Create data loader
        dataset = TensorDataset(
            torch.FloatTensor(X_train).to(self.device),
            torch.FloatTensor(y_train).to(self.device)
        )
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        for batch_X, batch_y in loader:
            self.optimizer.zero_grad()
            predictions = self.model(batch_X)
            loss = self.criterion(predictions, batch_y)

            loss.backward()
            # Gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            self.optimizer.step()

            total_loss += loss.item()
            n_batches += 1

        return total_loss / n_batches

    def validate(self, X_val, y_val, batch_size=32):
        self.model.eval()
        total_loss = 0
        n_batches = 0

        dataset = TensorDataset(
            torch.FloatTensor(X_val).to(self.device),
            torch.FloatTensor(y_val).to(self.device)
        )
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

        with torch.no_grad():
            for batch_X, batch_y in loader:
                predictions = self.model(batch_X)
                loss = self.criterion(predictions, batch_y)
                total_loss += loss.item()
                n_batches += 1

        return total_loss / n_batches

    def train_with_early_stopping(self, X_train, y_train, X_val, y_val,
                                  max_epochs=200, patience=20, batch_size=32,
                                  verbose=True):
        best_val_loss = float('inf')
        patience_counter = 0

        for epoch in range(max_epochs):
            train_loss = self.train_epoch(X_train, y_train, batch_size)
            val_loss = self.validate(X_val, y_val, batch_size)

            self.training_history['train_loss'].append(train_loss)
            self.training_history['val_loss'].append(val_loss)

            # Update learning rate
            self.scheduler.step(val_loss)

            # Early stopping check
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                # Save best model
                self.best_model_state = {k: v.cpu().clone() for k, v in self.model.state_dict().items()}
            else:
                patience_counter += 1

            if verbose and (epoch % 10 == 0 or epoch < 5):
                print(f"Epoch {epoch:3d}: Train Loss={train_loss:.6f}, Val Loss={val_loss:.6f}")

            if patience_counter >= patience:
                if verbose:
                    print(f"Early stopping at epoch {epoch}")
                break

        # Load best model
        if self.best_model_state is not None:
            self.model.load_state_dict(self.best_model_state)
            self.model.to(self.device)

        return self.training_history

    def predict(self, X_test):
        self.model.eval()
        with torch.no_grad():
            X = torch.FloatTensor(X_test).to(self.device)
            predictions = self.model(X).cpu().numpy()
        return predictions


print("GPUTrainer class defined successfully")

GPUTrainer class defined successfully


In [4]:
def _build_bitmasks(num_qubits, offset=0):
    masks = []
    for q in range(num_qubits):
        masks.append(1 << (offset + q))
    for i, j in combinations(range(num_qubits), 2):
        masks.append((1 << (offset + i)) | (1 << (offset + j)))
    return masks

def calc_observables(probs, bitmasks):
    exps = np.zeros(len(bitmasks), dtype=float)
    for basis_index, prob in enumerate(probs):
        if prob == 0.0:
            continue
        for obs_idx, mask in enumerate(bitmasks):
            parity = bin(mask & basis_index).count("1") % 2
            sign = -1.0 if parity else 1.0
            exps[obs_idx] += sign * prob
    return exps

def apply_amplitude_damping(probs, gamma_vec):
    damped = probs.astype(float, copy=True)
    if damped.size == 0:
        return damped
    indices = np.arange(damped.size, dtype=np.int64)
    for qubit, gamma in enumerate(gamma_vec):
        gamma = float(np.clip(gamma, 0.0, 1.0))
        if gamma <= 0.0:
            continue
        mask = ((indices >> qubit) & 1).astype(bool)
        if not mask.any():
            continue
        decay = gamma * damped[mask]
        damped[mask] -= decay
        targets = indices[mask] & ~(1 << qubit)
        np.add.at(damped, targets, decay)
    return damped

def build_params(f_b, arccos_zs):
    params = [f_b]
    params.extend(arccos_zs[:-1])
    params.extend(list(arccos_zs) * 4)
    return params

In [5]:
def apply_channel_coupling(observables_stack, coupling_pairs, coupling_strength):
    if not coupling_pairs or coupling_strength <= 0.0:
        return observables_stack
    mixed = observables_stack.copy()
    for i, j in coupling_pairs:
        oi = observables_stack[i]
        oj = observables_stack[j]
        mixed[i] = (1.0 - coupling_strength) * oi + coupling_strength * oj
        mixed[j] = (1.0 - coupling_strength) * oj + coupling_strength * oi
    return mixed

def qrc_circuit_independent(params, num_qubits):
    param_idx = 0
    for i in range(num_qubits):
        qml.RY(params[param_idx], wires=i)
        param_idx += 1
    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            qml.CNOT(wires=[i, j])
    for i in range(num_qubits):
        qml.RY(params[param_idx], wires=i)
        param_idx += 1
    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            qml.CNOT(wires=[i, j])
    for i in range(num_qubits):
        qml.RY(params[param_idx], wires=i)
        param_idx += 1
    for i in range(num_qubits):
        qml.RY(params[param_idx], wires=i)
        param_idx += 1
    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            qml.CNOT(wires=[i, j])
    for i in range(num_qubits):
        qml.RY(params[param_idx], wires=i)
        param_idx += 1
    return qml.probs(wires=range(num_qubits))

def qrc_circuit_entangled(params, coupling_angle, num_qubits, n_reservoirs, entangled_pair_indices):
    param_idx = 0
    total_qubits = num_qubits * n_reservoirs
    for res in range(n_reservoirs):
        offset = res * num_qubits
        for i in range(num_qubits):
            qml.RY(params[param_idx], wires=offset + i)
            param_idx += 1
        for i in range(num_qubits):
            for j in range(i + 1, num_qubits):
                qml.CNOT(wires=[offset + i, offset + j])
        for i in range(num_qubits):
            qml.RY(params[param_idx], wires=offset + i)
            param_idx += 1
        for i in range(num_qubits):
            for j in range(i + 1, num_qubits):
                qml.CNOT(wires=[offset + i, offset + j])
        for i in range(num_qubits):
            qml.RY(params[param_idx], wires=offset + i)
            param_idx += 1
        for i in range(num_qubits):
            qml.RY(params[param_idx], wires=offset + i)
            param_idx += 1
        for i in range(num_qubits):
            for j in range(i + 1, num_qubits):
                qml.CNOT(wires=[offset + i, offset + j])
        for i in range(num_qubits):
            qml.RY(params[param_idx], wires=offset + i)
            param_idx += 1
    
    if len(entangled_pair_indices) > 0:
        pair_count = len(entangled_pair_indices) // 2
        for p in range(pair_count):
            ctrl_idx = entangled_pair_indices[2 * p]
            tgt_idx = entangled_pair_indices[2 * p + 1]
            qml.CNOT(wires=[ctrl_idx, tgt_idx])
            qml.RY(coupling_angle, wires=tgt_idx)
            qml.CNOT(wires=[ctrl_idx, tgt_idx])
    
    return qml.probs(wires=range(total_qubits))

In [6]:
class QuantumReservoirPennyLane:
    def __init__(self, num_qubits=6, n_reservoirs=3, f_bs=None, b=-0.33,
                 ridge_alpha=3e-4, warmup_steps=5, amplitude_damping=None,
                 coupling_mode="independent", coupling_strength=0.0,
                 coupling_pairs=None):
        
        self.num_qubits = num_qubits
        self.n_reservoirs = n_reservoirs
        
        if f_bs is None:
            default_fb = [0.11, 0.1375, 0.12375]
            self.f_bs = default_fb[:self.n_reservoirs]
        else:
            self.f_bs = list(f_bs)
        
        if len(self.f_bs) != self.n_reservoirs:
            raise ValueError("Length of f_bs must match n_reservoirs.")
        
        self.b = b
        self.ridge_alpha = ridge_alpha
        self.warmup_steps = warmup_steps
        self.total_qubits = self.num_qubits * self.n_reservoirs
        
        self._bitmasks_local = _build_bitmasks(self.num_qubits, offset=0)
        self.n_observables = len(self._bitmasks_local)
        self._bitmasks_entangled = [
            _build_bitmasks(self.num_qubits, offset=res * self.num_qubits)
            for res in range(self.n_reservoirs)
        ]
        
        self.last_outputs = [np.zeros(self.num_qubits) for _ in range(self.n_reservoirs)]
        self.init_qrc_states = deepcopy(self.last_outputs)
        self.amplitude_damping = amplitude_damping
        self._gamma_map = self._prepare_damping(amplitude_damping)
        
        # Coupling config
        valid = {"independent", "channel", "entangled"}
        if coupling_mode not in valid:
            raise ValueError(f"coupling_mode must be in {valid}")
        self.coupling_mode = coupling_mode
        self.coupling_strength = float(np.clip(coupling_strength, 0.0, 1.0))
        
        if self.n_reservoirs < 2:
            self._coupling_pairs = []
        else:
            if coupling_pairs is None:
                self._coupling_pairs = [(i, i + 1) for i in range(self.n_reservoirs - 1)]
            else:
                self._coupling_pairs = coupling_pairs
        
        self._entangled_pair_indices = []
        if self._coupling_pairs:
            for src, dst in self._coupling_pairs:
                ctrl = src * self.num_qubits + (self.num_qubits - 1)
                tgt = dst * self.num_qubits
                self._entangled_pair_indices.extend([ctrl, tgt])
        
        self.ridge = Ridge(alpha=self.ridge_alpha)
        self._setup_devices()
    
    def _setup_devices(self):
        try:
            self.dev_independent = qml.device('lightning.gpu', wires=self.num_qubits)
            self.dev_entangled = qml.device('lightning.gpu', wires=self.total_qubits)
            self._device_type = "lightning.gpu"
        except:
            self.dev_independent = qml.device('lightning.qubit', wires=self.num_qubits)
            self.dev_entangled = qml.device('lightning.qubit', wires=self.total_qubits)
            self._device_type = "lightning.qubit"
    
    def _prepare_damping(self, amplitude_damping: Optional[Sequence[float]]):
        if amplitude_damping is None:
            return None
        if np.isscalar(amplitude_damping):
            gamma = float(np.clip(amplitude_damping, 0.0, 1.0))
            return [np.full(self.num_qubits, gamma, dtype=float) for _ in range(self.n_reservoirs)]
        arr = np.array(amplitude_damping, dtype=float)
        if arr.ndim == 1:
            if arr.size == 1:
                gamma = float(np.clip(arr[0], 0.0, 1.0))
                return [np.full(self.num_qubits, gamma, dtype=float) for _ in range(self.n_reservoirs)]
            if arr.size == self.n_reservoirs:
                return [np.full(self.num_qubits, float(np.clip(g, 0.0, 1.0)), dtype=float) for g in arr]
            if arr.size == self.num_qubits:
                base = np.clip(arr, 0.0, 1.0).astype(float)
                return [base.copy() for _ in range(self.n_reservoirs)]
            if arr.size == self.total_qubits:
                clipped = np.clip(arr, 0.0, 1.0).astype(float)
                return [
                    clipped[i * self.num_qubits:(i + 1) * self.num_qubits].copy()
                    for i in range(self.n_reservoirs)
                ]
        if arr.ndim == 2 and arr.shape == (self.n_reservoirs, self.num_qubits):
            clipped = np.clip(arr, 0.0, 1.0).astype(float)
            return [row.copy() for row in clipped]
        raise ValueError(
            "amplitude_damping must be None, a scalar, length-n_reservoirs, "  "length-num_qubits, length-total_qubits, or (n_reservoirs, num_qubits)."
        )
         

In [7]:
# Attach evolve + train/predict methods to QuantumReservoirPennyLane

def _evolve_independent(self, input_value):
    observables = []
    qnode = qml.QNode(lambda p: qrc_circuit_independent(p, self.num_qubits), self.dev_independent)
    for res_idx in range(self.n_reservoirs):
        zs = np.clip(self.last_outputs[res_idx], -1.0, 1.0)
        f_b = input_value * self.b * self.f_bs[res_idx]
        arccos_zs = np.arccos(zs) * self.b
        params_list = build_params(f_b, arccos_zs)
        probs = qnode(params_list)
        if self._gamma_map is not None:
            probs = apply_amplitude_damping(probs, self._gamma_map[res_idx])
        observables.append(calc_observables(probs, self._bitmasks_local))
    observables_stack = np.stack(observables)
    if self.coupling_mode == "channel" and getattr(self, "_coupling_pairs", []):
        observables_stack = apply_channel_coupling(observables_stack, self._coupling_pairs, self.coupling_strength)
    for res_idx in range(self.n_reservoirs):
        self.last_outputs[res_idx] = observables_stack[res_idx, :self.num_qubits]
    return observables_stack.reshape(-1)


def _evolve_entangled(self, input_value):
    param_chunks = []
    for res_idx in range(self.n_reservoirs):
        zs = np.clip(self.last_outputs[res_idx], -1.0, 1.0)
        f_b = input_value * self.b * self.f_bs[res_idx]
        arccos_zs = np.arccos(zs) * self.b
        param_chunks.append(build_params(f_b, arccos_zs))
    params_flat = np.concatenate(param_chunks)
    coupling_angle = float(self.coupling_strength) * np.pi
    qnode = qml.QNode(
        lambda p, a: qrc_circuit_entangled(p, a, self.num_qubits, self.n_reservoirs, self._entangled_pair_indices),
        self.dev_entangled
    )
    probs = qnode(params_flat, coupling_angle)
    if self._gamma_map is not None:
        gamma_flat = np.concatenate(self._gamma_map)
        probs = apply_amplitude_damping(probs, gamma_flat)
    observables = []
    for res_idx, bitmasks in enumerate(self._bitmasks_entangled):
        obs = calc_observables(probs, bitmasks)
        self.last_outputs[res_idx] = obs[:self.num_qubits]
        observables.append(obs)
    return np.concatenate(observables)


def evolve_qrc(self, input_value):
    if self.coupling_mode == "entangled":
        return self._evolve_entangled(input_value)
    return self._evolve_independent(input_value)


def reset_reservoirs(self):
    self.last_outputs = deepcopy(self.init_qrc_states)


def train(self, train_data):
    all_states, all_targets = [], []
    for series in train_data:
        self.reset_reservoirs()
        for _ in range(self.warmup_steps):
            _ = self.evolve_qrc(series[0])
        for t in range(len(series) - 1):
            state = self.evolve_qrc(series[t])
            if not np.all(np.isfinite(state)):
                state = np.nan_to_num(state, nan=0.0)
            all_states.append(state)
            all_targets.append(series[t + 1])
    X = np.array(all_states)
    y = np.array(all_targets)
    self.ridge.fit(X, y)


def predict(self, test_data, n_predict=20):
    n_test = test_data.shape[0]
    predictions = np.zeros((n_test, n_predict))
    for test_idx in range(n_test):
        series = test_data[test_idx]
        self.reset_reservoirs()
        for _ in range(self.warmup_steps):
            _ = self.evolve_qrc(series[0])
        for t in range(len(series) - 1):
            _ = self.evolve_qrc(series[t])
        last_value = series[-1]
        for step in range(n_predict):
            state = self.evolve_qrc(last_value)
            if not np.all(np.isfinite(state)):
                state = np.nan_to_num(state, nan=0.0)
            last_value = self.ridge.predict(state.reshape(1, -1))[0]
            predictions[test_idx, step] = last_value
    return predictions


# Bind to class
QuantumReservoirPennyLane._evolve_independent = _evolve_independent
QuantumReservoirPennyLane._evolve_entangled = _evolve_entangled
QuantumReservoirPennyLane.evolve_qrc = evolve_qrc
QuantumReservoirPennyLane.reset_reservoirs = reset_reservoirs
QuantumReservoirPennyLane.train = train
QuantumReservoirPennyLane.predict = predict

print("Bound evolve/train/predict methods to QuantumReservoirPennyLane")

Bound evolve/train/predict methods to QuantumReservoirPennyLane


In [8]:
# Assume QuantumReservoirPennyLane and helper functions (qrc_circuit_entangled, etc.) are defined elsewhere
class QuantumReservoirJIT(QuantumReservoirPennyLane):
    """
    Enhanced Quantum Reservoir with Catalyst JIT compilation.
    Inherits from QuantumReservoirPennyLane, adds JIT and neural readout support.
    """
    def __init__(self, *args, use_jit=True, readout_type='ridge', **kwargs):
        super().__init__(*args, **kwargs)
        self.use_jit = use_jit and CATALYST_AVAILABLE
        self.readout_type = readout_type
        self.neural_readout = None
        self.neural_trainer = None

        if self.use_jit:
            print(f"Setting up JIT-compiled circuits...")
            self._setup_jit_circuits()

    def _setup_jit_circuits(self):
        """Create JIT-compiled versions of quantum circuits"""
        if not CATALYST_AVAILABLE:
            print("Catalyst not available, skipping JIT setup")
            return

        try:
            if self.coupling_mode == "entangled":
                @qjit
                @qml.qnode(self.dev_entangled)
                def circuit_jit(params, coupling_angle):
                    return qrc_circuit_entangled(
                        params, coupling_angle, self.num_qubits,
                        self.n_reservoirs, self._entangled_pair_indices
                    )
                self.circuit_jit = circuit_jit
                print("JIT-compiled entangled circuit created")
            else:
                # For independent/channel modes, create per-reservoir JIT circuits
                @qjit
                @qml.qnode(self.dev_independent)
                def circuit_jit(params):
                    return qrc_circuit_independent(params, self.num_qubits)
                self.circuit_jit = circuit_jit
                print("JIT-compiled independent circuit created")
        except Exception as e:
            print(f"JIT compilation failed: {e}")
            print("Falling back to non-JIT mode")
            self.use_jit = False

    def evolve_qrc(self, input_value):
        """Override with JIT support"""
        if self.use_jit and hasattr(self, 'circuit_jit'):
            return self._evolve_jit(input_value)
        return super().evolve_qrc(input_value)

    def _evolve_jit(self, input_value):
        """JIT-optimized quantum evolution"""
        if self.coupling_mode == "entangled":
            return self._evolve_entangled_jit(input_value)
        return self._evolve_independent_jit(input_value)

    def _evolve_independent_jit(self, input_value):
        """JIT-compiled independent mode evolution"""
        observables = []

        for res_idx in range(self.n_reservoirs):
            zs = np.clip(self.last_outputs[res_idx], -1.0, 1.0)
            f_b = input_value * self.b * self.f_bs[res_idx]
            arccos_zs = np.arccos(zs) * self.b
            params_list = build_params(f_b, arccos_zs)

            # Use JIT-compiled circuit
            probs = np.array(self.circuit_jit(params_list))

            if self._gamma_map is not None:
                probs = apply_amplitude_damping(probs, self._gamma_map[res_idx])
            observables.append(calc_observables(probs, self._bitmasks_local))

        observables_stack = np.stack(observables)
        if self.coupling_mode == "channel":
            observables_stack = apply_channel_coupling(
                observables_stack, self._coupling_pairs, self.coupling_strength
            )

        for res_idx in range(self.n_reservoirs):
            self.last_outputs[res_idx] = observables_stack[res_idx, :self.num_qubits]

        return observables_stack.reshape(-1)

    def _evolve_entangled_jit(self, input_value):
        """JIT-compiled entangled mode evolution"""
        param_chunks = []
        for res_idx in range(self.n_reservoirs):
            zs = np.clip(self.last_outputs[res_idx], -1.0, 1.0)
            f_b = input_value * self.b * self.f_bs[res_idx]
            arccos_zs = np.arccos(zs) * self.b
            param_chunks.append(build_params(f_b, arccos_zs))

        params_flat = np.concatenate(param_chunks)
        coupling_angle = float(self.coupling_strength) * np.pi

        # Use JIT-compiled circuit
        probs = np.array(self.circuit_jit(params_flat, coupling_angle))

        if self._gamma_map is not None:
            gamma_flat = np.concatenate(self._gamma_map)
            probs = apply_amplitude_damping(probs, gamma_flat)

        observables = []
        for res_idx, bitmasks in enumerate(self._bitmasks_entangled):
            obs = calc_observables(probs, bitmasks)
            self.last_outputs[res_idx] = obs[:self.num_qubits]
            observables.append(obs)

        return np.concatenate(observables)

    def train_neural_readout(self, train_data, val_split=0.2, neural_config=None,
                             training_config=None):
        """
        Train neural network readout instead of Ridge regression.
        """
        if neural_config is None:
            neural_config = {
                'hidden_layers': [256, 128, 64],
                'dropout': 0.3,
                'readout_class': ImprovedNeuralReadout
            }

        if training_config is None:
            training_config = {
                'max_epochs': 200,
                'patience': 20,
                'batch_size': 32,
                'lr': 0.001,
                'weight_decay': 1e-4
            }

        print("Collecting quantum states...")
        all_states, all_targets = [], []

        for series_idx in tqdm(range(train_data.shape[0]), desc="Quantum Evolution", leave=False):
            series = train_data[series_idx]
            self.reset_reservoirs()

            # Warmup
            for _ in range(self.warmup_steps):
                _ = self.evolve_qrc(series[0])

            # Collect states
            for t in range(len(series) - 1):
                state = self.evolve_qrc(series[t])
                if not np.all(np.isfinite(state)):
                    state = np.nan_to_num(state, nan=0.0)
                all_states.append(state)
                all_targets.append(series[t + 1])

        X = np.array(all_states)
        y = np.array(all_targets)

        # Train/validation split
        n_train = int(len(X) * (1 - val_split))
        X_train, X_val = X[:n_train], X[n_train:]
        y_train, y_val = y[:n_train], y[n_train:]

        print(f"Training neural readout: {X_train.shape[0]} train, {X_val.shape[0]} val")

        # Create neural network
        ReadoutClass = neural_config.pop('readout_class', ImprovedNeuralReadout)
        self.neural_readout = ReadoutClass(input_size=X.shape[1], **neural_config)

        # Train on GPU
        self.neural_trainer = GPUTrainer(self.neural_readout, device=device,
                                         lr=training_config['lr'],
                                         weight_decay=training_config['weight_decay'])

        history = self.neural_trainer.train_with_early_stopping(
            X_train, y_train, X_val, y_val,
            max_epochs=training_config['max_epochs'],
            patience=training_config['patience'],
            batch_size=training_config['batch_size'],
            verbose=True
        )

        self.readout_type = 'neural'
        print("Neural readout training complete!")
        return history

    def predict(self, test_data, n_predict=20):
        """Override predict to use neural readout if available"""
        n_test = test_data.shape[0]
        predictions = np.zeros((n_test, n_predict))

        for test_idx in tqdm(range(n_test), desc="Predicting", leave=False):
            series = test_data[test_idx]
            self.reset_reservoirs()

            # Warmup
            for _ in range(self.warmup_steps):
                _ = self.evolve_qrc(series[0])

            # Initialize
            for t in range(len(series) - 1):
                _ = self.evolve_qrc(series[t])

            last_value = series[-1]

            # Predict future
            for step in range(n_predict):
                state = self.evolve_qrc(last_value)
                if not np.all(np.isfinite(state)):
                    state = np.nan_to_num(state, nan=0.0)

                # Use neural or ridge readout
                if self.readout_type == 'neural' and self.neural_readout is not None:
                    last_value = self.neural_trainer.predict(state.reshape(1, -1))[0]
                else:
                    last_value = self.ridge.predict(state.reshape(1, -1))[0]

                predictions[test_idx, step] = last_value

        return predictions


print("QuantumReservoirJIT class defined successfully")

QuantumReservoirJIT class defined successfully


In [9]:
def generate_damped_oscillator_data(n_train=10, n_test=4, n_points=26, seed=1):
    rng = np.random.Generator(np.random.PCG64(seed))
    t = np.linspace(0, 26, num=n_points)
    n_total = n_train + n_test
    oscis = 0.5 + rng.standard_normal(n_total) * 0.4
    data = np.zeros((n_total, n_points))
    for j in range(n_total):
        omega = oscis[j]
        data[j, :] = (
            np.exp(-0.05 * t) * 
            (np.cos(omega * t) + np.cos(omega * t / np.sqrt(2))) * 30 +
            rng.standard_normal(n_points) * 0.3
        )
    return data
data = generate_damped_oscillator_data(n_train=10, n_test=4, n_points=26, seed=1)
train_data = data[:10]
test_data = data[10:]

In [10]:
def run_hybrid_experiments(train_data, test_data, base_cfg, init_points=6, n_predict=20):
    """
    Compare Ridge, Neural, and JIT+Neural approaches.
    """
    results = {}

    # 1. Original Ridge (no JIT)
  
    print("1. BASELINE: Ridge Regression (No JIT)")

    cfg_ridge = dict(base_cfg, coupling_mode="entangled", coupling_strength=0.20)
    model_ridge = QuantumReservoirPennyLane(**cfg_ridge)

    start_time = time.time()
    model_ridge.train(train_data)
    preds_ridge = model_ridge.predict(test_data[:, :init_points], n_predict=n_predict)
    time_ridge = time.time() - start_time

    truth = test_data[:, init_points:].flatten()
    results['ridge'] = {
        'predictions': preds_ridge,
        'r2': r2_score(truth, preds_ridge.flatten()),
        'mse': mean_squared_error(truth, preds_ridge.flatten()),
        'nrmse': np.sqrt(mean_squared_error(truth, preds_ridge.flatten())) / (np.std(truth) + 1e-12),
        'time': time_ridge
    }
    print(f"R²={results['ridge']['r2']:.4f}, NRMSE={results['ridge']['nrmse']:.4f}, Time={time_ridge:.2f}s")

    # 2. Ridge with JIT
    
    print("2. Ridge + JIT Compilation")

    model_jit = QuantumReservoirJIT(**cfg_ridge, use_jit=True, readout_type='ridge')

    start_time = time.time()
    model_jit.train(train_data)
    preds_jit = model_jit.predict(test_data[:, :init_points], n_predict=n_predict)
    time_jit = time.time() - start_time

    results['jit'] = {
        'predictions': preds_jit,
        'r2': r2_score(truth, preds_jit.flatten()),
        'mse': mean_squared_error(truth, preds_jit.flatten()),
        'nrmse': np.sqrt(mean_squared_error(truth, preds_jit.flatten())) / (np.std(truth) + 1e-12),
        'time': time_jit,
        'speedup': time_ridge / time_jit if time_jit > 0 else 0
    }
    print(f"R²={results['jit']['r2']:.4f}, NRMSE={results['jit']['nrmse']:.4f}, Time={time_jit:.2f}s, Speedup={results['jit']['speedup']:.1f}x")

    # 3. Neural readout (no JIT)
   
    print("3. Neural Network Readout (No JIT)")

    model_neural = QuantumReservoirJIT(**cfg_ridge, use_jit=False, readout_type='neural')

    start_time = time.time()
    model_neural.train_neural_readout(train_data, val_split=0.2)
    preds_neural = model_neural.predict(test_data[:, :init_points], n_predict=n_predict)
    time_neural = time.time() - start_time

    results['neural'] = {
        'predictions': preds_neural,
        'r2': r2_score(truth, preds_neural.flatten()),
        'mse': mean_squared_error(truth, preds_neural.flatten()),
        'nrmse': np.sqrt(mean_squared_error(truth, preds_neural.flatten())) / (np.std(truth) + 1e-12),
        'time': time_neural
    }
    print(f"R²={results['neural']['r2']:.4f}, NRMSE={results['neural']['nrmse']:.4f}, Time={time_neural:.2f}s")

    # 4. HYBRID: JIT + Neural readout
    
    print("4. HYBRID: JIT + Neural Network Readout")
 
    model_hybrid = QuantumReservoirJIT(**cfg_ridge, use_jit=True, readout_type='neural')

    start_time = time.time()
    model_hybrid.train_neural_readout(train_data, val_split=0.2)
    preds_hybrid = model_hybrid.predict(test_data[:, :init_points], n_predict=n_predict)
    time_hybrid = time.time() - start_time

    results['hybrid'] = {
        'predictions': preds_hybrid,
        'r2': r2_score(truth, preds_hybrid.flatten()),
        'mse': mean_squared_error(truth, preds_hybrid.flatten()),
        'nrmse': np.sqrt(mean_squared_error(truth, preds_hybrid.flatten())) / (np.std(truth) + 1e-12),
        'time': time_hybrid,
        'speedup': time_ridge / time_hybrid if time_hybrid > 0 else 0
    }
    print(f"R²={results['hybrid']['r2']:.4f}, NRMSE={results['hybrid']['nrmse']:.4f}, Time={time_hybrid:.2f}s, Speedup={results['hybrid']['speedup']:.1f}x")

    return results


# Define base configuration for the quantum reservoir
base_cfg = dict(
    num_qubits=6,
    n_reservoirs=3,
    f_bs=[0.08, 0.10, 0.12],
    b=-0.40,
    warmup_steps=8,
    ridge_alpha=3e-4
)

# Assume train_data and test_data are pre-loaded numpy arrays
# Example placeholder data:
# train_data = np.random.rand(50, 20)
# test_data = np.random.rand(10, 26)

# Run comparison
hybrid_results = run_hybrid_experiments(train_data, test_data, base_cfg)

1. BASELINE: Ridge Regression (No JIT)
R²=0.6716, NRMSE=0.5731, Time=1759.05s
2. Ridge + JIT Compilation
Setting up JIT-compiled circuits...
JIT-compiled entangled circuit created


Predicting:   0%|          | 0/4 [00:00<?, ?it/s]

R²=0.6716, NRMSE=0.5731, Time=1751.86s, Speedup=1.0x
3. Neural Network Readout (No JIT)


Quantum Evolution:   0%|          | 0/10 [00:00<?, ?it/s]

Training neural readout: 200 train, 50 val
Epoch   0: Train Loss=406.549227, Val Loss=266.264210
Epoch   1: Train Loss=408.759818, Val Loss=263.442757
Epoch   2: Train Loss=352.694639, Val Loss=257.201138
Epoch   3: Train Loss=361.865069, Val Loss=249.217600
Epoch   4: Train Loss=332.444744, Val Loss=240.402662
Epoch  10: Train Loss=268.635587, Val Loss=155.939148
Epoch  20: Train Loss=194.337060, Val Loss=89.469435
Epoch  30: Train Loss=109.223185, Val Loss=50.885292
Epoch  40: Train Loss=63.192042, Val Loss=26.264825
Epoch  50: Train Loss=61.264717, Val Loss=21.837965
Epoch  60: Train Loss=50.611004, Val Loss=14.206161
Epoch  70: Train Loss=60.510548, Val Loss=10.295516
Epoch  80: Train Loss=46.800715, Val Loss=11.522503
Epoch  90: Train Loss=44.142929, Val Loss=8.294345
Early stopping at epoch 93
Neural readout training complete!


...

R²=-0.1367, NRMSE=1.0662, Time=1747.94s
4. HYBRID: JIT + Neural Network Readout
Setting up JIT-compiled circuits...
JIT-compiled entangled circuit created


Quantum Evolution:   0%|          | 0/10 [00:00<?, ?it/s]

Training neural readout: 200 train, 50 val
Epoch   0: Train Loss=438.207280, Val Loss=266.593384
Epoch   1: Train Loss=433.036024, Val Loss=264.096060
Epoch   2: Train Loss=356.488669, Val Loss=258.296865
Epoch   3: Train Loss=350.505057, Val Loss=250.825762
Epoch   4: Train Loss=322.246368, Val Loss=242.198568
Epoch  10: Train Loss=260.626744, Val Loss=152.121567
Epoch  20: Train Loss=164.842790, Val Loss=92.513235
Epoch  30: Train Loss=104.123186, Val Loss=50.352417
Epoch  40: Train Loss=81.171091, Val Loss=36.640297
Epoch  50: Train Loss=78.194746, Val Loss=23.248048
Epoch  60: Train Loss=65.821684, Val Loss=27.281194
Epoch  70: Train Loss=41.539595, Val Loss=8.825106
Epoch  80: Train Loss=54.953954, Val Loss=11.196260
Epoch  90: Train Loss=49.614410, Val Loss=8.101940
Early stopping at epoch 96
Neural readout training complete!


Predicting:   0%|          | 0/4 [00:00<?, ?it/s]

R²=0.2949, NRMSE=0.8397, Time=1759.10s, Speedup=1.0x
